**Spectral-Spatial Residual Network for Hyperspectral Image Classification: A 3-D Deep Learning Framework**

In [1]:
! pip install spectral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 6.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import spectral
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

In [3]:
class SSRN(nn.Module):
  def __init__(self):
    super(SSRN,self).__init__()
    self.FE1 = nn.Sequential(
        nn.Conv3d(in_channels=1,out_channels=24,kernel_size=(7,1,1),stride=(2,1,1)),
        nn.BatchNorm3d(24),
    )
    self.spe_conv1 = nn.Sequential(
        nn.Conv3d(in_channels=24,out_channels=24,kernel_size=(7,1,1),stride=(1,1,1),padding=(3,0,0)),
        nn.BatchNorm3d(24),
        nn.ReLU(inplace=True),
        nn.Conv3d(in_channels=24,out_channels=24,kernel_size=(7,1,1),stride=(1,1,1),padding=(3,0,0)),
        nn.BatchNorm3d(24),
        nn.ReLU(inplace=True),
    )
    self.spe_conv2 = nn.Sequential(
        nn.Conv3d(in_channels=24,out_channels=24,kernel_size=(7,1,1),padding=(3,0,0)),
        nn.BatchNorm3d(24),
        nn.ReLU(inplace=True),
        nn.Conv3d(in_channels=24,out_channels=24,kernel_size=(7,1,1),padding=(3,0,0)),
        nn.BatchNorm3d(24),
        nn.ReLU(inplace=True),
    )
    self.CF = nn.Sequential(
        nn.Conv3d(in_channels=24,out_channels=128,kernel_size=(12,1,1),stride=(1,1,1)),
        nn.BatchNorm3d(128),
    )
    self.FE2 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels=24,kernel_size=(3,3),stride=(1,1)),
        nn.BatchNorm2d(24),
    )
    self.spa_conv1 = nn.Sequential(
        nn.Conv2d(in_channels=24,out_channels=24,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
        nn.BatchNorm2d(24),
        nn.ReLU(inplace=True),
        nn.Dropout2d(p=0.3),
        nn.Conv2d(in_channels=24,out_channels=24,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
        nn.BatchNorm2d(24),
        nn.Dropout2d(p=0.3),
    )
    self.avgpool = nn.AvgPool2d(kernel_size=7)
    self.classification = nn.Linear(24,16)
  def forward(self,x):
    #print("输入数据的shape为：",x.shape)
    #x = torch.unsqueeze(x,dim=1)
    #print("输入数据扩展后的shape为：",x.shape)
    x = x.permute(0, 1, 4, 2, 3)
    FE1 = self.FE1(x)
    #print("FE1运行后的shape为：",FE1.shape)
    spe_conv1 = self.spe_conv1(FE1)
    spe_conv1_res = spe_conv1+FE1
    spe_conv2 = self.spe_conv2(spe_conv1_res)
    spe_conv2_res = spe_conv2+spe_conv1_res
    #print("spe_conv2_res的shape为：:",spe_conv2_res.shape)
    CF = self.CF(spe_conv2_res)
    #print("CF运行后的shape为：:",CF.shape)
    CF = torch.squeeze(CF,dim=2)
    #print("CF压缩后的shape为:",CF.shape)
    FE2 = self.FE2(CF)
    #print("FE2运行后的shape为：",FE2.shape)
    spa_conv1 = self.spa_conv1(FE2)
    spa_conv1_res = spa_conv1+FE2
    #print("spa_conv1_res的shape为：",spa_conv1_res.shape)
    avg = self.avgpool(spa_conv1_res)
    #print("spa_conv1_res在最大池化后的avg的shape为:",avg.shape)
    avg = torch.squeeze(avg,2)
    avg = torch.squeeze(avg,2)
    #print("avg在压缩后的shape为：",avg.shape)
    out = self.classification(avg)
    return out

原版的输入维度是100，本实验中为了方便和统一，将输入维度改为了30

In [7]:
# G2C-3DConv需要将维度设置大一些
# 第二个参数为classes
x = torch.randn(1, 1, 11, 11, 30)
net = SSRN()
y = net(x)
print(y.shape)

torch.Size([1, 16])
